In [1]:
import numpy as np

In [2]:
x = np.random.rand(10)

In [3]:
n_output = 2

In [4]:
x

array([0.46231763, 0.5744329 , 0.96603109, 0.26641787, 0.01928115,
       0.96659413, 0.68905617, 0.93951571, 0.72285003, 0.39574619])

In [5]:
w = np.random.rand(x.shape[0], n_output)

In [6]:
w

array([[0.41250582, 0.6025827 ],
       [0.03171325, 0.46517902],
       [0.98455096, 0.91351597],
       [0.49692695, 0.97482237],
       [0.11811645, 0.07744363],
       [0.80665282, 0.94443973],
       [0.16871911, 0.33852279],
       [0.08820113, 0.44921711],
       [0.60072112, 0.90086319],
       [0.1957843 , 0.58554952]])

In [7]:
b = np.random.rand(n_output)

In [8]:
y = x @ w + b

In [9]:
y

array([3.71730549, 4.33030509])

In [10]:
var = {
    'x @ w + b' : []
}

In [11]:
class LinearFunction(object):
    def __init__(self):
        self.c = {}

    def forward(self, x, w, b):
        self.c['x'] = x
        self.c['w'] = w
        self.c['b'] = b
        return x @ w + b
    
    def backward(self, dy):        
        return self.c['w'] @ dy, np.expand_dims(self.c['x'], 1) @ np.expand_dims(dy, 0), dy*1

In [12]:
class SigmoidFunction(object):
    def __init__(self):
        self.c = {}
        
    def forward(self, x):
        self.c['x'] = x
        return self.sigmoid(x)
    
    def sigmoid(self, x):
        return 1/(1 + np.exp(-x)) 
    
    def backward(self, dy):
        return dy * (self.sigmoid(self.c['x']) * (1 - self.sigmoid(self.c['x'])))

In [13]:
f = LinearFunction()
out1 = f.forward(x, w, b)
out1

array([3.71730549, 4.33030509])

In [14]:
s = SigmoidFunction()
out2 = s.forward(out1)
out2

array([0.9762771, 0.9870075])

In [15]:
d_out1 = s.backward(dy=1)

In [16]:
dx, dw, db = f.backward(dy=d_out1)

In [17]:
out2

array([0.9762771, 0.9870075])

In [18]:
target = np.array([0.5, 0.1])

In [19]:
class Network(object):
    def __init__(self, input_size, output_size):
        self.f = LinearFunction()
        self.s = SigmoidFunction()
        w = np.random.normal(loc=1, scale=1, size=(input_size, output_size))
        b = np.zeros(output_size)
        self.params = {'w': w, 'b': b}
    
    def forward(self, x):
        y1 = self.f.forward(x, self.params['w'], self.params['b'])
        y2 = self.s.forward(y1)
        return y2

    def backward(self, grad):
        dy2_dy1 = self.s.backward(dy=grad)
        return f.backward(dy=dy2_dy1)

class LossL1Function(object):
    def __init__(self):
        self.c = {}
        
    def forward(self, y, y_target):
        self.c['y'] = y
        self.c['y_target'] = y_target
        d = y_target - y
        self.c['d'] = d
        return np.mean(np.abs(d))
    
    def backward(self):
        weight = 1 / self.c['d'].shape[0]
        dl_dd = weight * ((self.c['d'] > 0) * 1.0 + (self.c['d'] < 0) * -1.0)
        dd_dy = dl_dd * -1
        
        return dd_dy

class GDOptimizer(object):    
    def __init__(self, params):
        self.params = params
    
    def step(self, dw, db, lr):
        self.params['w'] = self.params['w'] - dw * lr
        self.params['b'] = self.params['b'] - db * lr
        
def step(n, loss_object, optim, x, y_target, lr):
    y = n.forward(x)
    
    loss = loss_object.forward(y, y_target)
    dl_dy = loss_object.backward()
    
    _, dw, db = n.backward(grad=dl_dy)
    
    optim.step(dw, db, lr)
    return y, loss

In [20]:
def train():
    n = Network(input_size=10, output_size=2)
    loss = LossL1Function()
    optim = GDOptimizer(params=n.params)
    
    x = np.random.randn(10, )
    target = np.array([0.5, 0.1])
    lr = 0.1
    for i in range(10000):
        out, lss = step(n, loss, optim, x, target, lr=lr)
        if i % 100 == 0:
            print(out)
            
        if i % 1000 == 0:
            lr = lr - 0.001

train()

[0.99767089 0.8386201 ]
[0.99762965 0.55072435]
[0.99758693 0.19542187]
[0.99754265 0.09960471]
[0.99749672 0.09944205]
[0.99744904 0.10049287]
[0.99739952 0.10032732]
[0.99734804 0.10016243]
[0.99729448 0.09999819]
[0.99723873 0.09983397]
[0.99718064 0.09967039]
[0.99712067 0.09951679]
[0.99705811 0.10056015]
[0.99699279 0.10039766]
[0.99692451 0.1002358 ]
[0.99685306 0.10007457]
[0.99677823 0.09991364]
[0.99669976 0.09975305]
[0.99661739 0.09959307]
[0.99653082 0.09943371]
[0.99643972 0.10047529]
[0.99634472 0.10031021]
[0.99624454 0.10015196]
[0.99613874 0.0999943 ]
[0.99602682 0.09983665]
[0.99590826 0.0996796 ]
[0.99578244 0.09952315]
[0.99564867 0.10055754]
[0.99550617 0.10039835]
[0.99535408 0.10023976]
[0.99519138 0.10008178]
[0.99501878 0.09992117]
[0.99483342 0.09976703]
[0.99463381 0.09961346]
[0.99441826 0.09946046]
[0.99418477 0.10048437]
[0.99393103 0.1003287 ]
[0.9936543  0.10017362]
[0.9933513  0.10001912]
[0.99301816 0.09986469]
[0.99265014 0.09971076]
[0.99224595 0.09